In [ ]:
# https://medium.com/@mijanr/reading-and-plotting-historical-crypto-data-using-binance-api-with-python-c02a436c0307
import pandas as pd
import matplotlib.pyplot as plot

import datetime

useLocalOrRemoteData = False

# Define the start and end times for the data
end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(days=365)

# Convert the times to Unix timestamps in milliseconds
start_timestamp = int(start_time.timestamp() * 1000)
end_timestamp = int(end_time.timestamp() * 1000)

In [ ]:
from binance.client import Client
import binance
import os

if not useLocalOrRemoteData:
    api_key = os.environ.get('somekey1')
    api_secret = os.environ.get('somekey2')

    client = Client(api_key, api_secret)
    clientData = client.get_account()
    balances = clientData["balances"] # this is the table we're interested in
    # print(balances)

tickers = ['BTCUSDT', 'BTCBUSD','BTCEUR']#, 'SOLBUSD', 'SOLUSDT', 'XRPUSDT']
adabtc_trades = client.get_my_trades(symbol=tickers[0])

df = pd.DataFrame()

for ticker in tickers:
    data = client.get_my_trades(symbol=ticker)
    da = pd.DataFrame(data)
    df = pd.concat([df, da], axis=0, ignore_index=True, sort=False)
    df["value_euros"]= 0
df.set_index("id", inplace=True)

print(df)

In [ ]:
import json 

if useLocalOrRemoteData:
    # reading the data from the file 
    with open('account.json') as f: 
        data = f.read().replace('\'', '\"').replace("True", "\"True\"").replace("False", "\"False\"") 

    js = json.loads(data)
    balances = js["balances"]
    # print(balances)

In [ ]:
df = pd.DataFrame.from_dict(balances)
df.head()

# format and clean data
df["free"] = df["free"].transform(lambda x: float(x))
df["locked"]=df["locked"].transform(lambda x: float(x))
df = df[(df['free'] > 0) | (df['locked'] > 0)]
# print(df.head())

df["price"]=0.
df["value"]=0.
value = df.loc[df["asset"]=="BTC","free"]
print(df.reset_index(drop=True).head())

In [ ]:
# compute data

validAssets = ["BTC","ETH","ADA","DOT", "AXIS","BNB","SOL"]
assetsToRequest = []
for asset in df["asset"]:
    if not asset in validAssets:
        continue
    assetsToRequest.append(asset)

print("assets: ")
print(assetsToRequest)

def get_closing_price(klines):
    if klines:
        return klines[-1][4]  # Closing price is at index 4 in the kline list
    else:
        return None  # Handle cases where no data is found

from datetime import datetime

prices = {}
interval="1h"
startDateMs=int(datetime.strptime('1.12.2023 00:00','%d.%m.%Y %H:%M').timestamp() * 1000)
endDateMs=startDateMs
for ticker in assetsToRequest:
    try:
        klines = client.get_klines(symbol=ticker+"USDT", interval=interval, startTime=startDateMs, endTime=endDateMs)
        closing_price = float(get_closing_price(klines))
        prices[ticker] = closing_price
        assetRow = df.loc[df['asset'] == ticker]
        value = closing_price * (assetRow["free"] + assetRow["locked"])
        df.loc[df['asset'] == ticker, ['value']] = value
    except Exception as e:
        print(f"Error fetching price for {ticker}: {e}")


In [ ]:
# plot data

import plotly.graph_objects as go

filtered_df = df.loc[df["value"] > 0]
fig = go.Figure(data=[go.Table(
    header={ "values":list(filtered_df.columns),
                "fill_color":'paleturquoise',
                "align":'left'},
    cells={"values":filtered_df.transpose().values.tolist(),
               "fill_color":'lavender',
               "align":'left'})
])

fig.show()